<a href="https://colab.research.google.com/github/geopan2000/Loan-Approval-Prediction/blob/main/StoreSalesForecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Clone the GitHub repository
!git clone https://github.com/geopan2000/Store-Sales-Forecasting.git

# Change directory to the cloned repository
%cd Store-Sales-Forecasting

Cloning into 'Store-Sales-Forecasting'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 14 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (14/14), 229.89 KiB | 2.80 MiB/s, done.
/content/Store-Sales-Forecasting
